# FULCRUM - TOURNAMENT


In [33]:
import pandas as pd
import spacy
import re
import string
import numpy as np
import time
import requests
import json

In [ ]:
# alex
dataset_path = ''

embeddings_commercial_name_path = './data/'
embeddings_short_description_path = './data/'
embeddings_description_path = './data/'

# vali 
NAICS_KEYWORDS_PATH = './data/naics_summary_keywords.csv'

## Alex

In [34]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')


loaded_embeddings_commercial_name = np.load(embeddings_commercial_name_path)
loaded_embeddings_short_description = np.load(embeddings_short_description_path)
loaded_embeddings_description = np.load(embeddings_description_path)

data = pd.DataFrame(pd.read_csv(dataset_path))

#! Commercial name - cred

def round_1(company_name):
    sen = [company_name]
    sen_embeddings = model.encode(sen)

    results = cosine_similarity(
        [sen_embeddings[0]],
        loaded_embeddings_commercial_name[0:]
    )
    sim_value = results.max() # value
    label = data.iloc[results.argmax()]['naics_label'] # label
    return label

#! Short description

def round_3(short_description):
    sen = [short_description]
    sen_embeddings = model.encode(sen)

    results = cosine_similarity(
        [sen_embeddings[0]],
        loaded_embeddings_short_description[0:]
    )
    sim_value = results.max()
    label = data.iloc[results.argmax()]['naics_label']
    return label

#! Description

def round_5(description):
    sen = [description]
    sen_embeddings = model.encode(sen)

    results = cosine_similarity(
        [sen_embeddings[0]],
        loaded_embeddings_description[0:]
    )
    sim_value = results.max()
    label = data.iloc[results.argmax()]['naics_label']
    return label

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

C:\Users\valen\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\valen\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

FileNotFoundError: [Errno 2] No such file or directory: ''


## Vali - keywords for Round 2

In [29]:
pd_naics_keyw = pd.read_csv(NAICS_KEYWORDS_PATH)
pd_naics_keyw.head()

,Unnamed: 0,naics_code,naics_label,description,summary,keywords
0,0,111,Crop Production,Industries in the Crop Production subsector gr...,The Crop Production subsector comprises establ...,"Crop Production, farms, orchards, greenhouses,..."
1,1,112,Animal Production and Aquaculture,Industries in the Animal Production and Aquacu...,The Animal Production and Aquaculture subsecto...,"Animal Production, Aquaculture, Ranches, Farms..."
2,2,113,Forestry and Logging,Industries in the Forestry and Logging subsect...,The forestry and logging industries harvest ti...,"Forestry, Logging, Timber Production, Reforest..."
3,3,114,"Fishing, Hunting and Trapping","Industries in the Fishing, Hunting and Trappin...","The Fishing, Hunting and Trapping industry rel...","Fishing, Hunting, Trapping, Harvest, Wild Anim..."
4,4,115,Support Activities for Agriculture and Forestry,Industries in the Support Activities for Agric...,The Support Activities for Agriculture and For...,"Support Activities, Agriculture, Forestry, Sup..."


In [30]:
nlp = spacy.load("en_core_web_lg")

def remove_mentions_and_hashtags(text):
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#\w+", "", text)
    return text


def remove_numbers(text):
    text = re.sub(r"\d+", "", text)
    return text


def remove_punctuation(text):

    translator = str.maketrans("", "", string.punctuation)

    # Remove punctuation using the translation table
    text_without_punct = text.translate(translator)

    return text_without_punct


def remove_stopwords(text):
    filtered_sentence = []
    doc = nlp(text)
    for token in doc:
        if token.is_stop == False:
            filtered_sentence.append(token.text)
    return " ".join(filtered_sentence)


def clean_text(text,to_lemmatize:bool = True):

    # Standardize text
    # text = standardize_accented_chars(text)

    # Remove URLs
    text = re.sub(r"http\S+", "", text)

    # Remove mentions and hashtags
    text = remove_mentions_and_hashtags(text)

    # Lowercase
    text = text.lower()

    # Remove punctuation
    text = remove_punctuation(text)

    # Remove numbers
    text = remove_numbers(text)

    # Remove all the special characters
    text = re.sub(r"\W", " ", text)

    # Remove stopwords
    text = remove_stopwords(text)

    # Substituting multiple spaces with single space
    text = re.sub(r"\s+", " ", text, flags=re.I)

    if to_lemmatize:
        text = lemmatize(text)

    return text


def lemmatize(text):
    doc = nlp(text)
    text = " ".join([token.lemma_ for token in doc])
    return text

# Calculate similarity
def calculate_similarity(keywords, tokens):
    
    text1 = nlp(" ".join(keywords))
    text2 = nlp(" ".join(tokens))
    
    return text1.similarity(text2)


def predict_industry_label(keywords, industry_data):
    label_key = 'naics_label'
    code_key = 'naics_code'

#Precompute similarities if possible
    similarities = industry_data['tokens'].apply(lambda tokens: calculate_similarity(keywords, tokens))

    # Find index of the maximum similarity
    max_index = similarities.idxmax()
    max_similarity = similarities[max_index]
    
    # predicted_label = industry_data.at[max_index, code_key]
    # predicted_label = str(predicted_label) + " " + str(industry_data.at[max_index, label_key])
    predicted_code = str(industry_data.at[max_index, code_key])
    predicted_label = str(industry_data.at[max_index, label_key])

    return predicted_label, predicted_code, max_similarity


In [31]:

industry_data = pd_naics_keyw

# Preprocess industry descriptions
industry_data['processed_description'] = industry_data['keywords'].str.lower()
industry_data['processed_description'] = [clean_text(text) for text in industry_data['processed_description']]


print(industry_data['processed_description'][15])

# apply clean_text function to the column 'processed_description'
industry_data['processed_description'] = industry_data['processed_description'].apply(lambda x: clean_text(x))

# Tokenize and embed industry descriptions
industry_data['tokens'] = industry_data['processed_description'].apply(lambda x: [token.text for token in nlp(x)])

textile product mills nonapparel textile product sheet towel cut sew textile industry textile product purchase fabric cutting sewing


## API calls

In [32]:
base_url = 'http://116.202.111.229:8000'
api_key = 'VPLrVk4hSZMdGrW2wAP0GTpsV2Jsdx5Z'

headers = {
    'x-api-key': api_key
}

rounds = 5
current_round = 1

filename = 'extra_data.csv'
data_columns = ['commercial_name', 'business_tags', 'short_descripti', 'description', 'main_business_category', 'naics']

index = 0

# Get a new hint for current company or get the first hint for a new company after calling /evaluate/reset
while current_round != 6:
    response = requests.get(f"{base_url}/evaluate/hint", headers=headers)

    print(response.status_code, response.json())
    
    # Get the hint
    hint = response.json()['hint']

    # predict based off given hint
    time.sleep(1)
    
    final_answer = "abstain"
    
    if current_round == 3 or 4:
        # get the first 5 keywords from the text using spacy
        doc = nlp(hint)
        input_keywords = [chunk.text for chunk in doc.noun_chunks][:3]
    else:
        input_keywords = hint.split("|")
    
    print("Keywords: ", input_keywords)
    [final_answer, final_code, final_similarity] = predict_industry_label(input_keywords, industry_data)

    # Post your answer for current hint
    data = {
        'answer': final_answer
    }
    
    print("Guess: ", final_answer)
    print("Code: ", final_code)
    print("Similarity: ", final_similarity)
    
    response = requests.post(f"{base_url}/evaluate/answer", json=data, headers=headers)

    print(response.status_code, response.json())
    
    print("")
    
    current_round = current_round + 1
    time.sleep(1)

# Get hints about a new company
current_round = 1
index = index + 1

response = requests.get(f"{base_url}/evaluate/reset", headers=headers)

print(response.status_code, response.json())


200 {'company_id': 164, 'level': 1, 'hint': 'Happy Camper Wellness'}


Keywords:  ['Happy Camper Wellness']
Guess:  Sporting Goods, Hobby, Musical Instrument, Book, and Miscellaneous Retailers
Code:  459
Similarity:  0.33569560249402364
200 {'result': 'wrong', 'score': -50.0, 'answer': '624 Social Assistance'}

200 {'company_id': 164, 'level': 2, 'hint': 'Counseling and Coaching Services | Total Body Wellness | Mobile Fitness and Wellness Services | Individual Therapy | Career Cultivation | Licensed Clinical Social Worker | Licensed in Louisiana | Mindfulness | Anxiety & Depression | ADHD Parenting | Mental Illnesses | Therapy to Go Services | Consulting Services | Womens Health'}
Keywords:  ['Counseling', 'Coaching Services', 'Total Body Wellness']
Guess:  Social Assistance
Code:  624
Similarity:  0.5726204726115647
200 {'result': 'correct', 'score': 400.0, 'answer': '624 Social Assistance'}

200 {'company_id': 164, 'level': 3, 'hint': 'Therapy, counseling, coaching using mindfulness techniques to improve happiness at Happy Camper Wellness in Metairie, L